# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pathlib import Path # better file paths
from pyspark.sql.functions import countDistinct, col, when, lit, count
from pyspark.sql.functions import max as sparkMax #https://stackoverflow.com/questions/36924873/pyspark-column-is-not-iterable
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler # create numeric features
from pyspark.ml.feature import StringIndexer # convert Categorical variables to numeric Indexes
from pyspark.ml.feature import OneHotEncoderEstimator # one hot code categorical variables
from pyspark.ml import Pipeline # create a pipeline for different stages of preprocessing the data
from pyspark.ml.classification import LogisticRegression # a good starter algorithm for classification

In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName('Sparkify') \
    .getOrCreate()
spark

In [3]:
# what is the project folder?
import os; os.getcwd()

'/Users/jas/github/udacity-data-scientist-nanodegree-capstone-project'

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [4]:
# reading in data from local data folder which is gitignored due to large file size
event_data = Path.cwd() / "data" / "mini_sparkify_event_data.json"
df = spark.read.json(str(event_data))
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [5]:
print(df.columns) # this way prints horizontally

['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'method', 'page', 'registration', 'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId']


In [6]:
df.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Martha Tilston,Logged In,Colin,M,50,Freeman,277.89016,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Rockpools,200,1538352117000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
1,Five Iron Frenzy,Logged In,Micah,M,79,Long,236.09424,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1538331630000,8,Canada,200,1538352180000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
2,Adam Lambert,Logged In,Colin,M,51,Freeman,282.82730,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Time For Miracles,200,1538352394000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
3,Enigma,Logged In,Micah,M,80,Long,262.71302,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1538331630000,8,Knocking On Forbidden Doors,200,1538352416000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
4,Daft Punk,Logged In,Colin,M,52,Freeman,223.60771,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Harder Better Faster Stronger,200,1538352676000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30


# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

## Define Churn

* What column has Cancellation Confirmation? page?

In [7]:
df.groupby('page').count().sort('count', ascending=False).limit(50).toPandas()

,page,count
0,NextSong,228108
1,Home,14457
2,Thumbs Up,12551
3,Add to Playlist,6526
4,Add Friend,4277
5,Roll Advert,3933
6,Login,3241
7,Logout,3226
8,Thumbs Down,2546
9,Downgrade,2055


* How many total rows (in this sample dataset)?

In [8]:
num_rows = df.count()
num_rows

286500

* How many total users?

In [9]:
#num_users = df.select(countDistinct("userId")).toPandas().values[0]
num_users = df.select(countDistinct("userId")).toPandas().iloc[0][0]
num_users

226

In [10]:
num_users_cancelled = df.filter('page == "Cancellation Confirmation"').select(countDistinct("userId")).toPandas().iloc[0][0]
num_users_cancelled

52

In [11]:
round(num_users_cancelled/num_users * 100, 1) # percentage of users that cancelled

23.0

In [12]:
# actually update the data with churn value

* Just need to find the ids that churned and of course the rest are not churned

In [13]:
cancelled_ids = df.filter('page == "Cancellation Confirmation"').select("userId").distinct()
# Convert to list to be used to filter later
cancelled_ids = cancelled_ids.toPandas()['userId'].tolist()
cancelled_ids[:5]

['125', '51', '54', '100014', '101']

* When the userId matches a cancelled user than we provide a value of **1**, else it is **0**.

In [14]:
len(cancelled_ids)

52

In [15]:
df = df.withColumn("Churn", when((col("userId").isin(cancelled_ids)),lit('1')).otherwise(lit('0')))
df.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+-----+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|Churn|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+-----+
|      Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|       NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|    0|
|    Five Iron Frenzy|Logged In|    Micah|     M|           79|    L

In [16]:
df.groupby('Churn').agg(countDistinct("userId")).toPandas()

,Churn,count(DISTINCT userId)
0,0,174
1,1,52


## Explore Data

### How many columns and what type of data?

In [17]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- Churn: string (nullable = false)



In [18]:
## Descriptive stats
df.describe().toPandas()

,summary,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,Churn
0,count,228108,286500,278154,278154,286500,278154,228108,286500,278154,286500,286500,278154,286500,228108,286500,286500,278154,286500,286500
1,mean,551.0852017937219,None,None,None,114.41421291448516,None,249.11718197783722,None,None,None,None,1.535358834085557E12,1041.526554973822,Infinity,210.05459685863875,1.5409568898104714E12,None,59682.02278593872,0.1565933682373473
2,stddev,1217.7693079161374,None,None,None,129.76726201141085,None,99.23517921058324,None,None,None,None,3.291321616328068E9,726.7762634630834,NaN,31.50507848842202,1.5075439608187113E9,None,109091.94999910519,0.3634175920988577
3,min,!!!,Cancelled,Adelaida,F,0,Adams,0.78322,free,"Albany, OR",GET,About,1521380675000,1,ÃÂg ÃÂtti GrÃÂ¡a ÃÂsku,200,1538352117000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10)...",,0
4,max,ÃÂlafur Arnalds,Logged Out,Zyonna,M,1321,Wright,3024.66567,paid,"Winston-Salem, NC",PUT,Upgrade,1543247354000,2474,ÃÂau hafa sloppiÃÂ° undan ÃÂ¾unga myrkursins,404,1543799476000,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,99,1


## artist

* Who are the most popular artists?

In [19]:
df.groupby('artist').count().sort('count', ascending=False).limit(5).toPandas() 

,artist,count
0,None,58392
1,Kings Of Leon,1841
2,Coldplay,1813
3,Florence + The Machine,1236
4,Dwight Yoakam,1135


## auth

In [20]:
df.groupby('auth').count().sort('count', ascending=False).limit(5).toPandas() 

,auth,count
0,Logged In,278102
1,Logged Out,8249
2,Guest,97
3,Cancelled,52


## Create a function to do the same thing for each column

In [21]:
def topn_values(column_name, n):
    """
    Take a column name and find the most frequent values
    """
    return df.groupby(column_name).count().sort('count', ascending=False).limit(n).toPandas()

In [22]:
for column in df.columns:
    top_values = topn_values(column, 5)
    print(top_values, "\n")

artist  count
0                    None  58392
1           Kings Of Leon   1841
2                Coldplay   1813
3  Florence + The Machine   1236
4           Dwight Yoakam   1135 

         auth   count
0   Logged In  278102
1  Logged Out    8249
2       Guest      97
3   Cancelled      52 

  firstName  count
0    Payton   9632
1      None   8346
2     Riley   7970
3    Lucero   6880
4    Emilia   5732 

  gender   count
0      F  154578
1      M  123576
2   None    8346 

   itemInSession  count
0              0   3278
1              1   3125
2              2   3067
3              3   3013
4              4   2977 

   lastName  count
0  Campbell  14060
1      Reed   9284
2  Williams   8410
3      None   8346
4    Taylor   7230 

      length  count
0        NaN  58392
1  239.30730   1205
2  348.57751   1037
3  201.79546    908
4  655.77751    730 

  level   count
0  paid  228162
1  free   58338 

                                location  count
0     Los Angeles-Long Beach-Anaheim, C

### Findings
* Need to ignore users without a **userId**, since we can't aggregate their data across sessions, or maybe there are only a few hits within a session that are missing and in that case we can fill the data

* Need to exclude information that is not relevant for modeling such as first and last name, userId is all we need for now 

* How many user agents per user? Can we seperate out browser and OS?

* Can we extract the user's state from the location to create a new variable with less cardinality?

## Page

* Need to see more than 5 pages

In [23]:
topn_values('page', 20)

,page,count
0,NextSong,228108
1,Home,14457
2,Thumbs Up,12551
3,Add to Playlist,6526
4,Add Friend,4277
5,Roll Advert,3933
6,Login,3241
7,Logout,3226
8,Thumbs Down,2546
9,Downgrade,2055


## User agent

In [24]:
topn_values('userAgent', 20)

,userAgent,count
0,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",22751
1,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",19611
2,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",18448
3,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",17348
4,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,16700
5,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",15395
6,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",14598
7,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,10300
8,"""Mozilla/5.0 (iPad; CPU OS 7_1_2 like Mac OS X...",8912
9,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,8624


In [25]:
#https://stackoverflow.com/questions/17805464/using-regex-to-retrieve-browser-name

In [26]:
df.filter('LOWER(userAgent) rlike "(firefox|msie|chrome|safari)"').toPandas()['userAgent'].head().to_csv(sys.stdout) #https://stackoverflow.com/questions/29902714/print-very-long-string-completely-in-pandas-dataframe

,userAgent
0,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0
1,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"""
2,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0
3,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"""
4,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0


## Gender

* gender - we expect each user to have 1 value

In [27]:
df.groupBy('userId')\
.agg(countDistinct("gender"))\
.withColumnRenamed("count(DISTINCT gender)", 'gender_count')\
.filter('gender_count != 1').show()

+------+------------+
|userId|gender_count|
+------+------------+
|      |           0|
+------+------------+



* Is there a large difference between genders?

In [28]:
gender_tab = df.crosstab('Churn', 'Gender').toPandas()
gender_tab

,Churn_Gender,F,M,null
0,1,19168,25696,0
1,0,135410,97880,8346


In [29]:
gender_tab['F']

0     19168
1    135410
Name: F, dtype: int64

In [30]:
# Toy example

In [31]:
import plotly.express as px
df3 = px.data.tips()
df3.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [32]:

fig = px.bar(df3, y = "day", x="total_bill", color = "sex", title='Bar Chart' )
fig.show()

* Transform gender crosstab data from wide to long

In [33]:
fig = px.bar(df3,x="total_bill",  y = "day",  color = "sex", title='Bar Chart' )
fig.show()

In [34]:
# A better toy example
import plotly.graph_objects as go
animals=['giraffes', 'orangutans', 'monkeys']

fig = go.Figure(data=[
    go.Bar(name='SF Zoo', x=animals, y=[20, 14, 23]),
    go.Bar(name='LA Zoo', x=animals, y=[12, 18, 29])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

### Gender

In [35]:
churn = ['1', '0']

fig = go.Figure(data=[
    go.Bar(name='Male', x=churn, y=gender_tab['M']),
    go.Bar(name='Female', x=churn, y=gender_tab['F']),
    go.Bar(name='Other', x=churn, y=gender_tab['null'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

* Appears as though men disproportionately cancel more than women

* Wait a minute... data has not yet been aggregated to the user level

# Transform data to user level

## Feature engineering here as well mixed in

* how many unique artists did they listen to?
* how many unique songs? maybe churned users listened to less songs and were less active?
* how many unique sessions per user?
* how many 404 errors (not found)? Maybe frustrated users churned because they got too many errors?
* 307s are also a type of error
* Level - Since paid (p) is alphabetically higher than free (f), if we do max we will capture anyone who ever transitioned from free to paid, and of course if they were still free it will stay free

In [127]:
exprs = [\
   sparkMax(col('churn')).alias('churn')\
  ,sparkMax(col('Gender')).alias('gender')\
  ,sparkMax(col('level')).alias('subscription_level')\
  ,sparkMax(when(col("page") == 'Upgrade', 1).otherwise(0)).alias('page_upgraded')
  ,sparkMax(when(col("page") == 'Downgrade', 1).otherwise(0)).alias('page_downgraded')
  ,count(when(col("auth") == 'Logged In', True)).alias('auth_logged_in_cnt')\
  ,count(when(col("auth") == 'Logged Out', True)).alias('auth_logged_out_cnt')\
  ,count(when(col("auth") == 'Guest', True)).alias('auth_guest_cnt')\
  ,count(when(col("status") == '404', True)).alias('status_404_cnt')\
  ,count(when(col("status") == '307', True)).alias('status_307_cnt')\
  ,count(when(col("page") == 'Next Song', True)).alias('page_next_song_cnt')
  ,count(when(col("page") == 'Thumbs Up', True)).alias('page_thumbs_up_cnt')
  ,count(when(col("page") == 'Thumbs Down', True)).alias('page_thumbs_down_cnt')
  ,count(when(col("page") == 'Add to Playlist', True)).alias('page_playlist_cnt')
  ,count(when(col("page") == 'Add Friend', True)).alias('page_friend_cnt')
  ,count(when(col("page") == 'Roll Advert', True)).alias('page_roll_ad_cnt')
  ,count(when(col("page") == 'Logout', True)).alias('page_logout_cnt')
  ,count(when(col("page") == 'Help', True)).alias('page_help_cnt')
  ,countDistinct('artist').alias('artist_cnt')\
  ,countDistinct('song').alias('song_cnt')\
  ,countDistinct('sessionId').alias('session_cnt')\
]

user_df = df.groupBy('userId')\
.agg(*exprs)

# Should have already removed rows with no userIds earlier
# Remove data with null values - needs to be added to pipeline
user_df = user_df.where(col("gender").isNotNull()) #only 1 value is null and can't assume a value on a heuristic

user_df.toPandas().head()

AnalysisException: "cannot resolve '`userId`' given input columns: [features];;\n'Aggregate ['userId], ['userId, max('churn) AS churn#11575, max('Gender) AS gender#11577, max('level) AS subscription_level#11579, max(CASE WHEN ('page = Upgrade) THEN 1 ELSE 0 END) AS page_upgraded#11581, max(CASE WHEN ('page = Downgrade) THEN 1 ELSE 0 END) AS page_downgraded#11583, count(CASE WHEN ('auth = Logged In) THEN true END) AS auth_logged_in_cnt#11585, count(CASE WHEN ('auth = Logged Out) THEN true END) AS auth_logged_out_cnt#11587, count(CASE WHEN ('auth = Guest) THEN true END) AS auth_guest_cnt#11589, count(CASE WHEN ('status = 404) THEN true END) AS status_404_cnt#11591, count(CASE WHEN ('status = 307) THEN true END) AS status_307_cnt#11593, count(CASE WHEN ('page = Next Song) THEN true END) AS page_next_song_cnt#11595, count(CASE WHEN ('page = Thumbs Up) THEN true END) AS page_thumbs_up_cnt#11597, count(CASE WHEN ('page = Thumbs Down) THEN true END) AS page_thumbs_down_cnt#11599, count(CASE W

In [ ]:
user_df.count() # same as original userId count near top 

In [ ]:
user_df.where(col("userId").isNull()).show() # we are going to ignore users without a userId since we can't aggregate

In [ ]:
### rename columns (don't need to anymore)
#newcolnames = ['userId', 'gender', 'churn']
# for c, n in zip(user_df.columns, newcolnames): #https://stackoverflow.com/questions/34077353/how-to-change-dataframe-column-names-in-pyspark
#     user_df = user_df.withColumnRenamed(c, n)
#user_df.columns 

In [ ]:
user_df.show()

In [ ]:
user_df.filter('auth_Guest_cnt != 0').show()

* Okay it appears that when a user logs out we don't see there associated userId, so this data cannot be used.. also need need to remove this NULL or empty user Id

# EDA Visualizations continued...

In [ ]:
gender_tab = user_df.crosstab('churn', 'gender').toPandas()

churn = ['1', '0']

fig = go.Figure(data=[
    go.Bar(name='Male', x=churn, y=gender_tab['M']),
    go.Bar(name='Female', x=churn, y=gender_tab['F']),
    go.Bar(name='Other', x=churn, y=gender_tab['null'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

* Still appears that gender does differ significantly in each group

### Subscription level

In [ ]:
user_df.crosstab('churn', 'subscription_level').toPandas()


In [ ]:
subscription_level_tab = user_df.crosstab('churn', 'subscription_level').toPandas()

churn = ['1', '0']

fig = go.Figure(data=[
    go.Bar(name='free', x=churn, y=subscription_level_tab['free']),
    go.Bar(name='paid', x=churn, y=subscription_level_tab['paid']),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

* appears a larger proportion of free churned vs non churned users (0)

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
exprs = [\
   sparkMax(col('churn')).alias('churn')\
  ,sparkMax(col('Gender')).alias('gender')\
  ,sparkMax(col('level')).alias('subscription_level')\
  ,sparkMax(when(col("page") == 'Upgrade', 1).otherwise(0)).alias('page_upgraded')
  ,sparkMax(when(col("page") == 'Downgrade', 1).otherwise(0)).alias('page_downgraded')
  ,count(when(col("auth") == 'Logged In', True)).alias('auth_logged_in_cnt')\
  ,count(when(col("auth") == 'Logged Out', True)).alias('auth_logged_out_cnt')\
  ,count(when(col("auth") == 'Guest', True)).alias('auth_guest_cnt')\
  ,count(when(col("status") == '404', True)).alias('status_404_cnt')\
  ,count(when(col("status") == '307', True)).alias('status_307_cnt')\
  ,count(when(col("page") == 'Next Song', True)).alias('page_next_song_cnt')
  ,count(when(col("page") == 'Thumbs Up', True)).alias('page_thumbs_up_cnt')
  ,count(when(col("page") == 'Thumbs Down', True)).alias('page_thumbs_down_cnt')
  ,count(when(col("page") == 'Add to Playlist', True)).alias('page_playlist_cnt')
  ,count(when(col("page") == 'Add Friend', True)).alias('page_friend_cnt')
  ,count(when(col("page") == 'Roll Advert', True)).alias('page_roll_ad_cnt')
  ,count(when(col("page") == 'Logout', True)).alias('page_logout_cnt')
  ,count(when(col("page") == 'Help', True)).alias('page_help_cnt')
  ,countDistinct('artist').alias('artist_cnt')\
  ,countDistinct('song').alias('song_cnt')\
  ,countDistinct('sessionId').alias('session_cnt')\
]

final_user_df = df.groupBy('userId')\
.agg(*exprs)
final_user_df.toPandas().head()

## Correlation Analysis

* Are any of the current variables highly correlated?

In [ ]:
# toy example
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),),
        (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
        (Vectors.dense([6.0, 7.0, 0.0, 8.0]),),
        (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)]
df = spark.createDataFrame(data, ["features"])

r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

r2 = Correlation.corr(df, "features", "spearman").head()
print("Spearman correlation matrix:\n" + str(r2[0]))

In [ ]:
df.head()

## Convert categorical variables to stringIndex

* Need to do this on a list of categorical variables later in a pipeline

In [ ]:
user_df.printSchema()

In [ ]:
user_df.groupBy("gender").count().show() # Only 1 null so dropped it

In [ ]:
user_df.groupBy("subscription_level").count().show()

* need to do it one categorical variable at a time for now

In [ ]:
indexer = StringIndexer(inputCol="gender", outputCol="gender_Index")
model_df = indexer.fit(user_df).transform(user_df)
model_df.select(["gender","gender_Index"]).show(5)

In [ ]:
indexer = StringIndexer(inputCol="subscription_level", outputCol="subscription_level_Index")
model_df = indexer.fit(model_df).transform(model_df)
model_df.select(["subscription_level","subscription_level_Index"]).show(5)

## One hot code categorical features

In [ ]:
encoder = OneHotEncoderEstimator(inputCols=["gender_Index", "subscription_level_Index"],
                                 outputCols=["gender_Vec", "subscription_level_Vec"])
model_df = encoder.fit(model_df).transform(model_df)
model_df.select(["gender","gender_Index","gender_Vec","subscription_level","subscription_level_Index","subscription_level_Vec"]).show(5)

## Create numeric features

In [ ]:
print(model_df.columns)

In [ ]:
numeric_columns = ['auth_logged_in_cnt', 'auth_guest_cnt', 'status_404_cnt', 'page_next_song_cnt']

In [ ]:
assembler = VectorAssembler(inputCols=numeric_columns, outputCol="NumericFeatures")
model_df = assembler.transform(model_df)
model_df.head()
model_df.select("NumericFeatures", "churn").show(truncate=False)

In [ ]:
from pyspark.ml.stat import Correlation
r1 = Correlation.corr(model_df, "NumericFeatures").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

## Normalize the numeric features

In [ ]:
scaler = Normalizer(inputCol="NumericFeatures", outputCol="ScaledNumericFeatures")
model_df = scaler.transform(model_df)
model_df.head(2)

In [ ]:
model_df.select("ScaledNumericFeatures", "churn").show()

* Combine the categorical variables from earlier..

In [ ]:
assembler = VectorAssembler(inputCols=all_columns, outputCol="AllFeatures")
model_df = assembler.transform(model_df)
model_df.select("AllFeatures", "churn").show(truncate=False)

## Pipeline

* Put it all together in a clean pipeline

In [ ]:
numeric_columns = ['auth_logged_in_cnt', 'auth_guest_cnt', 'status_404_cnt', 'page_next_song_cnt']
categorical = ['gender_Vec','subscription_level_Vec'] # TO DO: can convert to a for loop or anonymous function
all_columns = ['ScaledNumericFeatures'] + categorical

In [ ]:
assembler_numeric = VectorAssembler(inputCols=numeric_columns, outputCol="NumericFeatures")
scaler = Normalizer(inputCol="NumericFeatures", outputCol="ScaledNumericFeatures")
# Maybe this can be done in parallel like in sci-kit learn?
indexer_gender = StringIndexer(inputCol="gender", outputCol="gender_Index")
indexer_subscription_level = StringIndexer(inputCol="subscription_level", outputCol="subscription_level_Index")
encoder_onehot = OneHotEncoderEstimator(inputCols=["gender_Index", "subscription_level_Index"],
                                 outputCols=["gender_Vec", "subscription_level_Vec"])
assembler_all = VectorAssembler(inputCols=all_columns, outputCol="features") # churn already specified as label
lr = LogisticRegression(maxIter=10, regParam=0.001)

In [ ]:
pipeline = Pipeline(stages=[assembler_numeric, scaler, indexer_gender, indexer_subscription_level, encoder_onehot, assembler_all, lr])
pipeline

In [ ]:
user_df = user_df.withColumn("label", col("churn")) # need to be named as such

In [ ]:
model = pipeline.fit(user_df)

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [ ]:
# baseline model for comparison

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.